In [1]:
import numpy as np
import librosa
import io
import soundfile as sf
from moviepy.editor import VideoFileClip
from tqdm import tqdm
import pickle as pk
import os
import tensorflow as tf
# from tensorflow.keras.saving import register_keras_serializable
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
# real_audio_dir = (
#     r"H:\.shortcut-targets-by-id\1jH_pc6mMj0Iu8wLS1r0vggMWpVElJvOU\SIH2024_DATASET\REAL"
# )
# fake_audio_dir = (
#     r"H:\.shortcut-targets-by-id\1jH_pc6mMj0Iu8wLS1r0vggMWpVElJvOU\SIH2024_DATASET\FAKE"
# )

In [3]:
# with open(
#     r"H:\.shortcut-targets-by-id\1jH_pc6mMj0Iu8wLS1r0vggMWpVElJvOU\SIH2024_DATASET\real_files.pkl",
#     "rb",
# ) as f:
#     real_files = pk.load(f)

# with open(
#     r"H:\.shortcut-targets-by-id\1jH_pc6mMj0Iu8wLS1r0vggMWpVElJvOU\SIH2024_DATASET\fake_files.pkl",
#     "rb",
# ) as f:
#     fake_files = pk.load(f)

In [4]:
# len(real_files), len(fake_files)

In [5]:
# real_files = real_files[:2000]
# fake_files = fake_files[:2000]

In [6]:
# fake_files = fake_files[: len(real_files)]

In [7]:
# len(real_files), len(fake_files)

In [8]:
# def extract_features(file_path):
#     try:
#         # Load the video file
#         video_clip = VideoFileClip(file_path)
#         audio = video_clip.audio
#         fps = audio.fps
#         audio_samples = np.array(
#             list(audio.iter_frames(fps=fps, dtype="float32"))
#         ).flatten()
#         buffer = io.BytesIO()
#         sf.write(buffer, audio_samples, fps, format="wav")
#         buffer.seek(0)
#         x, sr = librosa.load(buffer, sr=None)
#         mfccs = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=20)

#         return mfccs

#     except Exception as e:
#         print(f"Error encountered while parsing file: {file_path}, {e}")
#         return None


# def load_data(real_dir, fake_dir):
#     labels = []
#     features = []

#     # Load real audios
#     for file_name in real_files:
#         file_path = os.path.join(real_dir, file_name)
#         mfccs = extract_features(file_path)
#         if mfccs is not None:
#             features.append(mfccs)
#             labels.append(0)  # 0 for REAL

#     # Load fake audios
#     for file_name in fake_files:
#         file_path = os.path.join(fake_dir, file_name)
#         mfccs = extract_features(file_path)
#         if mfccs is not None:
#             features.append(mfccs)
#             labels.append(1)  # 1 for FAKE

#     return np.array(features), np.array(labels)

In [9]:
# def extract_frame_features(file_path, frame_duration=1.0):
#     try:
#         video_clip = VideoFileClip(file_path)
#         audio = video_clip.audio
#         fps = audio.fps
#         audio_samples = np.array(
#             list(audio.iter_frames(fps=fps, dtype="float32"))
#         ).flatten()
#         buffer = io.BytesIO()
#         sf.write(buffer, audio_samples, fps, format="wav")
#         buffer.seek(0)
#         x, sr = librosa.load(buffer, sr=None)

#         # Split audio into frames of 'frame_duration' seconds
#         frame_length = int(frame_duration * sr)
#         frames = [
#             librosa.feature.mfcc(y=x[i : i + frame_length], sr=sr, n_mfcc=20)
#             for i in range(0, len(x), frame_length)
#             if i + frame_length <= len(x)
#         ]

#         return frames  # Returns list of MFCCs for each frame

#     except Exception as e:
#         print(f"Error processing file {file_path}: {e}")
#         return None

In [10]:
def extract_frame_features(file_path, frame_duration=1.0):
    video_clip = VideoFileClip(file_path)
    audio = video_clip.audio
    fps = audio.fps
    audio_samples = np.array(
        list(audio.iter_frames(fps=fps, dtype="float32"))
    ).flatten()
    buffer = io.BytesIO()
    sf.write(buffer, audio_samples, fps, format="wav")
    buffer.seek(0)
    x, sr = librosa.load(buffer, sr=None)

    # Split audio into frames of 'frame_duration' seconds
    frame_length = int(frame_duration * sr)
    frames = []
    timestamps = []

    for i in range(0, len(x), frame_length):
        if i + frame_length <= len(x):
            # Extract MFCCs for each frame and store the timestamp
            frame_mfcc = librosa.feature.mfcc(y=x[i: i + frame_length], sr=sr, n_mfcc=20)
            frames.append(frame_mfcc)
            timestamp = i / sr  # Convert index to seconds
            timestamps.append(timestamp)

    return frames, timestamps

In [11]:
# def load_data(real_dir, fake_dir, real_files, fake_files):
#     labels, features = [], []

#     # Load real audio frames with progress bar
#     print("Loading real audio files:")
#     for file_name in tqdm(real_files, desc="Processing Real Files"):
#         file_path = os.path.join(real_dir, file_name)
#         frame_features, timestamps = extract_frame_features(file_path)
#         if frame_features:
#             features.extend(frame_features)
#             labels.extend([0] * len(frame_features))  # Label 0 for REAL

#     # Load fake audio frames with progress bar
#     print("Loading fake audio files:")
#     for file_name in tqdm(fake_files, desc="Processing Fake Files"):
#         file_path = os.path.join(fake_dir, file_name)
#         frame_features = extract_frame_features(file_path)
#         if frame_features:
#             features.extend(frame_features)
#             labels.extend([1] * len(frame_features))  # Label 1 for FAKE

#     # Convert to numpy arrays
#     features = np.array(features)
#     labels = np.array(labels)

#     # Shuffle the data
#     indices = np.arange(len(features))
#     np.random.shuffle(indices)
#     features = features[indices]
#     labels = labels[indices]

#     return features, labels

In [12]:
# X, y = load_data(real_audio_dir, fake_audio_dir, real_files, fake_files)
# X = X[..., np.newaxis]

In [13]:
# with open("X_for_dl_2000.pkl", "wb") as f:
#     pk.dump(X, f)
# with open("y_for_dl_2000.pkl", "wb") as f:
#     pk.dump(y, f)

In [14]:
with open("X_for_dl_2000.pkl", "rb") as f:
    X = pk.load(f)
with open("y_for_dl_2000.pkl", "rb") as f:
    y = pk.load(f)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=30
)

## TCN

In [16]:
# model = models.Sequential(
#     [
#         layers.Conv1D(
#             64,
#             kernel_size=3,
#             dilation_rate=1,
#             padding="causal",
#             activation="relu",
#             input_shape=(X.shape[1], X.shape[2]),
#         ),
#         layers.Conv1D(
#             128, kernel_size=3, dilation_rate=2, padding="causal", activation="relu"
#         ),
#         layers.Conv1D(
#             256, kernel_size=3, dilation_rate=4, padding="causal", activation="relu"
#         ),
#         layers.GlobalAveragePooling1D(),
#         layers.Dropout(0.5),
#         layers.Dense(64, activation="relu"),
#         layers.Dense(2, activation="softmax"),
#     ]
# )

In [17]:
from tensorflow.keras import models, layers

model = models.Sequential(
    [
        layers.Conv1D(
            64,
            kernel_size=3,
            dilation_rate=1,
            padding="causal",
            activation="relu",
            input_shape=(X.shape[1], X.shape[2]),
        ),
        layers.BatchNormalization(),
        layers.Conv1D(
            128, kernel_size=3, dilation_rate=2, padding="causal", activation="relu"
        ),
        layers.BatchNormalization(),
        layers.Conv1D(
            256, kernel_size=3, dilation_rate=4, padding="causal", activation="relu"
        ),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling1D(),
        layers.Dropout(0.5),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(2, activation="softmax"),
    ]
)

In [18]:
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [19]:
checkpoint = ModelCheckpoint(
    "model/best_model.keras", monitor="val_loss", save_best_only=True
)
early_stopping = EarlyStopping(monitor="val_loss", patience=3)

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 20, 64)         │        16,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 20, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 20, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 20, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 20, 256)        │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 20, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 174,978 (683.51 KB)

 Trainable params: 174,082 (680.01 KB)

 Non-trainable params: 896 (3.50 KB)

In [21]:
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint],
)

Epoch 1/50
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.6385 - loss: 0.6350 - val_accuracy: 0.6710 - val_loss: 0.6011
Epoch 2/50
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.6680 - loss: 0.6062 - val_accuracy: 0.6838 - val_loss: 0.5800
Epoch 3/50
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.6856 - loss: 0.5882 - val_accuracy: 0.7069 - val_loss: 0.5591
Epoch 4/50
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.6969 - loss: 0.5731 - val_accuracy: 0.7187 - val_loss: 0.5497
Epoch 5/50
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.7038 - loss: 0.5649 - val_accuracy: 0.7303 - val_loss: 0.5353
Epoch 6/50
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.7127 - loss: 0.5569 - val_accuracy: 0.7343 - val_loss: 0.5330
Epoch 7/50
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.7198 - loss: 0.5478 - val_accuracy: 0.7102 - val_loss: 0.5598
Epoch 8/50
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.7239 - loss: 0

In [ ]:
model = tf.keras.models.load_model("model/TCN.keras")

In [36]:
# model = tf.keras.models.load_model("model/best_model.keras")
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Print classification report
print(classification_report(y_test, y_pred_labels, target_names=["REAL", "FAKE"]))
print(confusion_matrix(y_test, y_pred_labels))

500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

        REAL       0.89      0.62      0.73      7920
        FAKE       0.71      0.92      0.80      8072

    accuracy                           0.77     15992
   macro avg       0.80      0.77      0.77     15992
weighted avg       0.80      0.77      0.77     15992

[[4910 3010]
 [ 623 7449]]


In [37]:
def test_on_video(file_path, frame_duration=1.0):
    # Load the trained model
    model = tf.keras.models.load_model("model/TCN.keras")

    # Extract features and timestamps for each frame in the new video
    frames, timestamps = extract_frame_features(file_path, frame_duration)

    if frames is None or timestamps is None:
        print("No frames extracted.")
        return

    # Reshape frames for model input
    frames = np.array(frames)[..., np.newaxis]

    # Predict on each frame
    predictions = model.predict(frames)
    pred_labels = np.argmax(predictions, axis=1)

    # Store deepfake frames, their timestamps, and frame indices
    deepfake_frames = []
    deepfake_timestamps = []
    deepfake_indices = []

    # Identify deepfake frames
    for i, label in enumerate(pred_labels):
        if label == 1:  # If the label is FAKE
            deepfake_frames.append(frames[i])
            deepfake_timestamps.append(timestamps[i])
            deepfake_indices.append(i)

    if not deepfake_frames:
        print("No deepfake frames detected in the video.")
        return

    # Analyze deepfake frames
    print(f"Found {len(deepfake_frames)} deepfake frames:")
    for i, (timestamp, index) in enumerate(zip(deepfake_timestamps, deepfake_indices)):
        print(f"Frame {index + 1} at {timestamp:.2f}s: FAKE")


# Example usage
test_video_path = r"REAL\ajqslcypsw.mp4"  # Replace with your test video path
test_on_video(test_video_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
Found 8 deepfake frames:
Frame 1 at 0.00s: FAKE
Frame 2 at 1.00s: FAKE
Frame 3 at 2.00s: FAKE
Frame 4 at 3.00s: FAKE
Frame 5 at 4.00s: FAKE
Frame 8 at 7.00s: FAKE
Frame 11 at 10.00s: FAKE
Frame 15 at 14.00s: FAKE


In [25]:
# def test_on_video(file_path, frame_duration=1.0):
#     # Load the trained model
#     model = tf.keras.models.load_model("model/best_model.keras")

#     # Extract features for each frame in the new video
#     frames = extract_frame_features(file_path, frame_duration)

#     if frames is None:
#         print("No frames extracted.")
#         return

#     # Reshape frames for model input
#     frames = np.array(frames)[..., np.newaxis]

#     # Predict on each frame
#     predictions = model.predict(frames)
#     pred_labels = np.argmax(predictions, axis=1)

#     # Output results for each frame
#     for i, label in enumerate(pred_labels):
#         status = "REAL" if label == 0 else "FAKE"
#         print(f"Frame {i+1}: {status}")


# # Example usage
# test_video_path = r"REAL\bddjdhzfze.mp4"  # Replace with your test video path
# test_on_video(test_video_path)

In [26]:
# @register_keras_serializable()
# class AudioModel(tf.keras.Model):
#     def __init__(self, input_shape):
#         super(AudioModel, self).__init__()
#         self.input_shape = input_shape  # Store the input shape
#         # Define the model layers
#         self.conv1 = layers.Conv2D(
#             32, kernel_size=(3, 3), activation="relu", input_shape=input_shape
#         )
#         self.conv2 = layers.Conv2D(64, kernel_size=(3, 3), activation="relu")
#         self.pool = layers.MaxPooling2D(pool_size=(2, 2))
#         self.dropout1 = layers.Dropout(0.25)

#         self.reshape = layers.Reshape((64, -1))
#         self.gru = layers.Bidirectional(layers.GRU(128, return_sequences=False))

#         self.dense1 = layers.Dense(128, activation="relu")
#         self.dropout2 = layers.Dropout(0.5)
#         self.dense2 = layers.Dense(2, activation="softmax")

#     def call(self, inputs):
#         # Forward pass through the layers
#         x = self.conv1(inputs)
#         x = self.conv2(x)
#         x = self.pool(x)
#         x = self.dropout1(x)

#         x = self.reshape(x)
#         x = self.gru(x)

#         x = self.dense1(x)
#         x = self.dropout2(x)
#         return self.dense2(x)

#     def get_config(self):
#         config = super(AudioModel, self).get_config()
#         config.update(
#             {"input_shape": self.input_shape}  # Include input shape in config
#         )
#         return config

#     @classmethod
#     def from_config(cls, config):
#         # Create a model instance from the config
#         input_shape = config.pop("input_shape")  # Extract input_shape from config
#         return cls(input_shape)  # Create an instance of the model


# # Function to create and compile the model
# def create_model(input_shape):
#     model = AudioModel(input_shape)
#     model.compile(
#         optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
#     )
#     return model


# # Example usage
# input_shape = (
#     64,
#     40,
#     1,
# )  # Adjust based on your data (e.g., (n_mfccs, time_steps, channels))

In [27]:
# model = create_model(input_shape)
# model.summary()

In [28]:
# checkpoint = ModelCheckpoint(r"models/dl_model.keras", monitor="val_loss", save_best_only=True, verbose=1)
# early_stopping = EarlyStopping(monitor="val_loss", patience=5, verbose=1)

# history = model.fit(
#     X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping]
# )

In [29]:
# model.save(r"models/dl_model.keras", overwrite=True)
# print("Model saved successfully.")

In [30]:
# # Ensure to import keras properly
# import tensorflow as tf
# from tensorflow import keras


# # Function to load the model
# def load_model(model_path):
#     try:
#         # Load the model from the specified path
#         model = keras.models.load_model(model_path)
#         print("Model loaded successfully.")
#         return model
#     except Exception as e:
#         print(f"Error loading model: {e}")
#         return None

In [31]:
# model_path = r"models/dl_model.keras"

# # Load the model
# loaded_model = load_model(model_path)